# Black ❤ Jupyter
  
Date: 2019-07-22  
Author: Ricardo  
Categories: tips, formatting, pandas  
Tags: pandas, formatting, black, code, comprehension, list  
<!--eofm-->

After nearly a year coding in Python (although not consistently), I started trying code formatters and discovered they were more useful than I thought. Here are some reason why.
<!--more-->

First of all I must admit these tips may be biased by the fact I use [Jupyter Notebooks](https://jupyterlab.readthedocs.io/en/stable/) quite a lot. I think they are an incredible tool for learning although they may also instill some bad coding habits.

Before talking about [black](https://github.com/psf/black) lets go through the step to install it and use it with Jupyter Notebooks.

## Installing black and using it with Jupyter

Luckily for us, there is a Jupyterlab [extension](https://github.com/ryantam626/jupyterlab_code_formatter) for code formatting. To get started you only need 4 commands if you are using **conda**:

```bash
conda install -c conda-forge black
jupyter labextension install @ryantam626/jupyterlab_code_formatter
conda install -c conda-forge jupyterlab_code_formatter
jupyter serverextension enable --py jupyterlab_code_formatter
```

If you are using **pip** you need to install the one of the supported formatters (black in this case) and then do:

```bash
jupyter labextension install @ryantam626/jupyterlab_code_formatter
pip install jupyterlab_code_formatter
jupyter serverextension enable --py jupyterlab_code_formatter
```

Finally, what really helps me using black continuously, is having a handy shortcut to run it in the current cell. In my case I use **ctrl + k**, you can add it to your jupyterlab shortcuts pasting the following in your shortcuts settings editor:

```json
{

    "shortcuts": [
    {
                "command": "jupyterlab_code_formatter:black",
                "keys": [
                    "Ctrl K"
                ],
                "selector": ".jp-Notebook.jp-mod-editMode"
            }
        ]
}
```

Now lets go back to code ❗️

## Pandas method chaining

I believe one of those bad habits that Jupyter creates is writing code that doesn't look very clean when used outside the notebook. One case can be when using [method-chaining](https://tomaugspurger.github.io/method-chaining) in Pandas, which is the currently promoted style when cleaning data. Lets look at an example.

In [46]:
def read_messages(path):
    df = (
        pd.read_csv(
            path,
            delimiter=";",
            dtype={"id": "int",
                   "type": "category",
                   "title": "str",
                   "text": "str"})
        .drop(["Unnamed: 0"], axis=1)
        # dates
        .assign(date=lambda x: pd.to_datetime(x["date"], format="%Y-%m-%d %H:%M:%S"),
                date_start=lambda x: pd.to_datetime(x["date_start"], format="%Y-%m-%d %H:%M:%S"),
                date_end=lambda x: pd.to_datetime(x["date_end"], format="%Y-%m-%d %H:%M:%S"))
        # others
        .assign(txt_length=lambda x: x.text.str.len(),
                word_number=lambda x: x.text.str.split().apply(len),
                date=lambda x: pd.to_datetime(x["date"].apply(lambda y: y.strftime("%Y-%m-%d")), format="%Y-%m-%d",),
                hour=lambda x: x["date"].apply(lambda y: y.hour)))

    return df

That is probably the best we would do if we were writing code and experimenting with data cleaning functions. Even though it looks quite readable (I think when you get used to method chaining it gets a lot easier to read the code), there are a lot of parentheses, indents and long functions.

This is how it looks when we apply [black](https://github.com/python/black) to it.

In [47]:
def read_messages(path):
    df = (
        pd.read_csv(
            path,
            delimiter=";",
            dtype={"id": "int", "type": "category", "title": "str", "text": "str"},
        )
        .drop(["Unnamed: 0"], axis=1)
        # dates
        .assign(
            date=lambda x: pd.to_datetime(x["date"], format="%Y-%m-%d %H:%M:%S"),
            date_start=lambda x: pd.to_datetime(
                x["date_start"], format="%Y-%m-%d %H:%M:%S"
            ),
            date_end=lambda x: pd.to_datetime(
                x["date_end"], format="%Y-%m-%d %H:%M:%S"
            ),
        )
        .assign(
            txt_length=lambda x: x.text.str.len(),
            word_number=lambda x: x.text.str.split().apply(len),
            date=lambda x: pd.to_datetime(
                x["date"].apply(lambda y: y.strftime("%Y-%m-%d")), format="%Y-%m-%d"
            ),
            hour=lambda x: x["date"].apply(lambda y: y.hour),
        )
    )

    return df

That looks a lot better, and most importantly, it gets incredibily easier to debug when you get an error. At first I thought I would be using black only after I finished writing my functions, but now it's actually the other way around, by constantly keeping the same clean code style I can spot the bugs faster.

## Long, messy functions

Now we are going to simulate a function with many parameters. This example was taken from the great blog [Mouse vs Python](http://www.blog.pythonlibrary.org/2019/07/16/intro-to-black-the-uncompromising-python-code-formatter/) by [Mike Driscoll](https://twitter.com/driscollis).

In [48]:
def long_func(x, param_one=None, param_two=[], param_three={}, param_four=None, param_five="", param_six=123456):
    print(f"{param_three['first']}")

The function takes a lot of (unnecessary) arguments and then prints one of the values of the `dictionary` *param_three*. But what happens if we pass a `list` instead is that we get an error, and we can see that with such a long function, if the parameters were more difficult to debug and read this could turn out in one of these bugs that look abvious to solve but you can't find which parameter you are messing with.

In [49]:
long_func(2, param_one=None, param_two=[], param_three=['cat'], param_four=None, param_five="", param_six=123456)

TypeError: list indices must be integers or slices, not str

This is how it looks like after applying black, a lot better!

In [50]:
def long_func(
    x,
    param_one=None,
    param_two=[],
    param_three={},
    param_four=None,
    param_five="",
    param_six=123456,
):
    print(f"{param_three['first']}")

Even if we wanted to do type annotations it would be easy to read:

In [51]:
from typing import List, Dict, Tuple


def long_func(
    x,
    param_one: int = None,
    param_two: List = [],
    param_three: Dict = {},
    param_four: Tuple = None,
    param_five: str = "",
    param_six: float = 123456.34,
) -> None:

    print(f"{param_three['first']}")

Another thing I love about black is that it won't reformat the code if there are syntax mistakes, and sometimes this has let me spot the before executing a slow function, which would have made me notice the error only during runtime.

For example in the function above if we had used `-->` for the return value annotation instead of `->` (this has happened to me very often), black would not have changed the original (unreadable) function.

In [52]:
def long_func(x, param_one: int = None, param_two: List = [], param_three: Dict = {},
              param_four: Tuple = None, param_five: str = "",
              param_six: float = 123456.34) --> bool:
                                    ##     ^^^^ this is not correct
                                    ##     so this cell won't be formatted
                                    ##     when applying black!
                                    ##
                                    ##     I'm using 'bool' because I added
                                    ##     a return True at the end.
                            
    print(f"{param_three['first']}")

    return True

SyntaxError: invalid syntax (<ipython-input-52-eac28ca8f8ab>, line 3)

## Bonus

Since we are talking about formatting and debugging, I would like to show something I found recently, and it is the advantages of splitting code in multiple lines.

To illustrate it I am going to use list comprehensions. The example is taken from this [blog post](https://treyhunner.com/2019/03/abusing-and-overusing-list-comprehensions-in-python/) by [@treyhunner](https://twitter.com/treyhunner), which I highly recommend.

We are going to write a function to get the factors of a number using a list comprehension.

In [53]:
def get_factors(dividend):
    """Return a list of all factors of the given number."""
    return [n for n in range(1, dividend + 1) if dividend % n == 0]

In [54]:
get_factors(134)

[1, 2, 67, 134]

But if we happen to mess the variables and we try to add an **int** and a **string**:

In [55]:
def get_factors(dividend):
    """Return a list of all factors of the given number."""

    num = "ops, this was supposed to be a number, not a string"

    return [n for n in range(1, dividend + num) if dividend % n == 0]

In [56]:
get_factors(134)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Now we are getting an error, but we don't know exactly in what part of the comprehensions it is (this is just for illustration, in this case it is very easy to see where the bug is).

However, if we had sepparated the comprehension in multiple lines, we would now exactly where the error is:

In [57]:
# fmt: off
def get_factors(dividend):
    """Return a list of all factors of the given number."""
    
    num = "ops, this was supposed to be a number, not a string"

    return [
        n
        for n in range(1, dividend + num)
        if dividend % n == 0
    ]
# fmt: on

get_factors(134)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

We can see the interpreter points at the exact line (9) where the problematic variable is. This can also be useful when creating **pandas pivot tables**, since it's easy to make mistakes the first times you do it.

Lastly, notice the comments `# fmt: off` and `# fmt: on`. Those are because black would reformat that function to a one-line comprehension 😂, and with that comments we can avoid it.

In [58]:
# fmt: off
def get_factors(dividend):
    """Return a list of all factors of the given number."""
    return [
        n
        for n in range(1, dividend + 1)
        if dividend % n == 0
    ]
# fmt: on

In the end, formatting is something very personal. After starting to use black I found the consistency of the formatting really helped me be more productive and also understand the code faster we I went back to it.

The trend with Jupyter notebooks is quite clear, more and more tools keep appearing and more people are using them. One of the use cases for notebooks nowadays is teaching, and there's no doubt why, but being able to split your functions and outputs to make it clearer does not mean unreadable code is allowed. I believe many tutorials would be easier to follow only if the code were kept consistent and readable across notebooks.

To sum up, I found using a code formatter while writing code in Jupyter made me a lot more productive and it's something I will thank myself for  if I ever need to reread an old notebook.

**I hope you enjoyed it!**